In [2]:
import sys
sys.path.append("../../..")

import pandas as pd
import random
import numpy as np

train_data = pd.read_csv("../../../data/a0910/train.csv")
valid_data = pd.read_csv("../../../data/a0910/valid.csv")
test_data = pd.read_csv("../../../data/a0910/test.csv")

sample_size = 6

def fake_train_data(train_data, valid_data, test_data, s):
    all_data = pd.concat([train_data, valid_data, test_data], axis=0)
    existing_all_users = all_data['user_id'].unique()
    existing_train_users = train_data['user_id'].unique()
    new_train_data = []
    for train_user in existing_train_users:
        user_data = train_data[train_data['user_id'] == train_user]

        num_rows = len(user_data)
        if num_rows < s:
            random_rows = list(range(num_rows))
            s = num_rows
        else:
            random_rows = random.sample(range(num_rows), s)

        fake_user_ids = []
        for _ in range(s):
            fake_user_id = max(existing_all_users) + 1
            while fake_user_id in existing_all_users:
                fake_user_id += 1
            existing_all_users = np.append(existing_all_users, fake_user_id)
            fake_user_ids.append(fake_user_id)

        pos = [(user_data.iloc[i]['score'] - 0.5) * 2 for i in random_rows]
        row_count = 0
        for index, row in user_data.iterrows():
            user_id = row['user_id']
            item_id = row['item_id']
            score = row['score']

            data_point = {
                'user_id': user_id,
                'item_id': item_id,
                'score': score,
                'user_id_pair': fake_user_ids,
                'pos': pos,
                'fake': 0
            }
            new_train_data.append(data_point)

            for i in range(s):   
                if row_count == random_rows[i]:
                    fake_score = 1 - score
                    fake_data_point = {
                        'user_id': fake_user_ids[i],
                        'item_id': item_id,
                        'score': fake_score,
                        'user_id_pair': [user_id] * s,
                        'pos': [- pos[i]] * s,
                        'fake': 1
                    }
                    new_train_data.append(fake_data_point)
                else:
                    fake_data_point = {
                        'user_id': fake_user_ids[i],
                        'item_id': item_id,
                        'score': score,
                        'user_id_pair': [user_id] * s,
                        'pos': [- pos[i]] * s,
                        'fake': 1
                    }
                    new_train_data.append(fake_data_point)
            row_count += 1

    new_train_data_df = pd.DataFrame(new_train_data)
    shuffled_data = new_train_data_df.sample(frac=1).reset_index(drop=True)
    return shuffled_data, max(existing_all_users)

train_data, user_num = fake_train_data(train_data, valid_data, test_data, sample_size)
train_data.to_csv("../../../data/a0910/" + str(sample_size) + "_fake_train.csv", index=False)

In [1]:
# coding: utf-8
# 2021/3/23 @ tongshiwei
import sys
sys.path.append("../../..")

import logging
from EduCDM import CIRT
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import random
import numpy as np

batch_size = 256
sample_size = 3

train_data = pd.read_csv("../../../data/a0910/" + str(sample_size) + "_fake_train.csv")
valid_data = pd.read_csv("../../../data/a0910/valid.csv")
test_data = pd.read_csv("../../../data/a0910/test.csv")


def transform(x, y, z, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)

def transform_train_data(x, y, z, f, p, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32),
        torch.tensor(f, dtype=torch.int64),
        torch.tensor(p, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)



valid, test = [
    transform(data["user_id"], data["item_id"], data["score"], batch_size)
    for data in [valid_data, test_data]
]


user_id_pair = []
pos = []
for i, s in train_data.iterrows():
    codes1 = eval(s['user_id_pair'])
    codes2 = eval(s['pos'])
    user_id_pair.append(codes1)
    pos.append(codes2)
    
train = transform_train_data(train_data["user_id"], train_data["item_id"], train_data["score"], user_id_pair, pos, batch_size)
# print(train_data["pos"])
# print(max(train_data['user_id'].unique()))


/data/zhengz/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import sys
sys.path.append("../../..")

import logging
from EduCDM import CIRT
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import random
import numpy as np

logging.getLogger().setLevel(logging.INFO)

user_num = max(train_data['user_id'].unique())
# print(user_num)
cdm = CIRT(user_num + 1, 17747, a_range=1, zeta=0.3)

cdm.train(train, valid, epoch=20, device="cuda")
cdm.save("cirt_a0910.params")

cdm.load("cirt_a0910.params")
rmse, mae, auc, accuracy = cdm.eval(test)
print("rmse: %.6f, mae: %.6f, auc: %.6f, accuracy: %.6f" % (rmse, mae, auc, accuracy))

Epoch 0: 100%|██████████| 2908/2908 [00:36<00:00, 79.02it/s]


[Epoch 0] LogisticLoss: 0.768744


evaluating: 100%|██████████| 101/101 [00:00<00:00, 291.04it/s]


[Epoch 0] rmse: 0.490710, mae: 0.410496, auc: 0.551550, accuracy: 0.647622


Epoch 1: 100%|██████████| 2908/2908 [00:38<00:00, 76.04it/s] 


...

[Epoch 19] LogisticLoss: 0.451409


evaluating: 100%|██████████| 101/101 [00:00<00:00, 196.96it/s]
INFO:root:save parameters to cirt_a0910.params
INFO:root:load parameters from cirt_a0910.params


[Epoch 19] rmse: 0.436485, mae: 0.343824, auc: 0.746368, accuracy: 0.716551


evaluating: 100%|██████████| 218/218 [00:00<00:00, 239.07it/s]


rmse: 0.437926, mae: 0.346090, auc: 0.745606, accuracy: 0.715208


In [4]:
import sys
sys.path.append("../../..")

import logging
from EduCDM import CIRT
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import random
import numpy as np

logging.getLogger().setLevel(logging.INFO)

user_num = max(train_data['user_id'].unique())
# print(user_num)
cdm = CIRT(user_num + 1, 17747, a_range=1, zeta=0.3)

cdm.train(train, valid, epoch=20, device="cuda")
cdm.save("cirt_a0910.params")

cdm.load("cirt_a0910.params")
rmse, mae, auc, accuracy = cdm.eval(test)
print("rmse: %.6f, mae: %.6f, auc: %.6f, accuracy: %.6f" % (rmse, mae, auc, accuracy))

Epoch 0:   1%|          | 24/2181 [00:00<00:19, 110.79it/s]

Epoch 0: 100%|██████████| 2181/2181 [00:22<00:00, 99.07it/s] 


[Epoch 0] LogisticLoss: 0.768978, Count: 562846


evaluating: 100%|██████████| 101/101 [00:00<00:00, 253.73it/s]


[Valid Loss] 0.720974
[Epoch 0] rmse: 0.493900, mae: 0.410246, auc: 0.545596, accuracy: 0.647114


...

[Epoch 5] LogisticLoss: 0.564941, Count: 380279


evaluating: 100%|██████████| 101/101 [00:00<00:00, 254.99it/s]


[Valid Loss] 0.601874
[Epoch 5] rmse: 0.451772, mae: 0.381778, auc: 0.692413, accuracy: 0.688589


Epoch 6:  22%|██▏       | 474/2181 [00:04<00:17, 98.96it/s] 


KeyboardInterrupt: 

In [7]:
batch_size = 256
def transform(x, y, z, batch_size, **params):
    x = x.to_numpy()
    y = y.to_numpy()
    z = z.to_numpy()
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)


test_data = pd.read_csv('../../../data/a0910/test.csv')
user_records_count = pd.read_csv('../../../data/a0910/user_records_count.csv')
test_data = test_data.merge(user_records_count, on='user_id', how='inner')

test1_data = test_data[test_data['count'] < 200]
test2_data = test_data[test_data['count'] >= 200]

test1, test2 = [
    transform(data["user_id"], data["item_id"], data["score"], batch_size)
    for data in [test1_data, test2_data]
]
test1, test2

(<torch.utils.data.dataloader.DataLoader at 0x7efcd72ce8c0>,
 <torch.utils.data.dataloader.DataLoader at 0x7efcd72cecb0>)

In [8]:
cdm.load("cirt_a0910.params")
rmse, mae, auc, accuracy = cdm.eval(test1)
print("rmse: %.6f, mae: %.6f, auc: %.6f, accuracy: %.6f" % (rmse, mae, auc, accuracy))

cdm.load("cirt_a0910.params")
rmse, mae, auc, accuracy = cdm.eval(test2)
print("rmse: %.6f, mae: %.6f, auc: %.6f, accuracy: %.6f" % (rmse, mae, auc, accuracy))

INFO:root:load parameters from cirt_a0910.params
evaluating: 100%|██████████| 104/104 [00:00<00:00, 288.09it/s]
INFO:root:load parameters from cirt_a0910.params


rmse: 0.454218, mae: 0.368573, auc: 0.729730, accuracy: 0.692842


evaluating: 100%|██████████| 115/115 [00:00<00:00, 262.70it/s]

rmse: 0.422614, mae: 0.325706, auc: 0.757075, accuracy: 0.735485
